In [2]:
import pyspark
import re
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
sc = pyspark.SparkContext("local", "WordCount")

25/03/04 03:54:15 WARN Utils: Your hostname, jovan-laptop resolves to a loopback address: 127.0.1.1; using 192.168.8.175 instead (on interface wlp3s0)
25/03/04 03:54:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/04 03:54:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load text files
doc1 = sc.textFile("pg75509.txt")
doc2 = sc.textFile("pg75510.txt")

In [5]:
doc1.take(10)

Py4JJavaError: An error occurred while calling o23.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovan/repos/DATA.ML.370/pg75509.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:210)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scal

In [ ]:
# Tokenize, clean, and filter words
words1 = doc1.flatMap(lambda line: line.split())\
             .map(lambda word: re.sub(r'[^a-zA-Z\s]', '', word).lower().strip())\
             .filter(lambda word: word and not word.isdigit())

words2 = doc2.flatMap(lambda line: line.split())\
             .map(lambda word: re.sub(r'[^a-zA-Z\s]', '', word).lower().strip())\
             .filter(lambda word: word and not word.isdigit())

In [ ]:
print(words1.take(10))

In [ ]:
# Count word occurrences
counts1 = words1.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
counts2 = words2.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)